# Algorithm

1. Background segmentation (GrabCut)
2. Background fitting (Gaussian)
3. Brightness and contrast adjustment

# Background Segmentation and Fitting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from scipy import optimize

def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / 4 / stddev)**2)

path_original = r'Folder of ORIGINAL images'
files_original = os.listdir(path_original)

## Please refer to https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html for the function of marked images
path_marked = r'Folder of MARKED images'
files_marked = os.listdir(path_marked)

for f in range(len(files_original)):
    
## Part I: Background Segmentation (GrabCut)

    file_original = files_original[f]
    file_marked = files_marked[f]
    
    imgpath_original = path_original+'/'+file_original
    img = cv2.imread(imgpath_original)

    imgpath_marked = path_marked+'/'+file_marked
    newmask = cv2.imread(imgpath_marked,0)
    
    ## GaussianBlur is to make the image less noisy if necessary
    # img = cv2.GaussianBlur(img,(5,5), 0.5)
    # newmask = cv2.GaussianBlur(newmask,(5,5), 0.5)

    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    x_0 = ## x value of the top left corner of the rect
    y_0 = ## y value of the top left corner of the rect
    x_1 = ## x value of the bottom right corner of the rect
    y_1 = ## y value of the bottom right corner of the rect
    rect = (x_0,y_0,x_1,y_1)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT) # 5 is the number of iterations

    mask[newmask == 0] = 0
    mask[newmask == 255] = 1 ## use the marked image to correct the mask
    mask, bgdModel, fgdModel = cv2.grabCut(img,mask,None,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_MASK)
    mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img_fgd = img*mask[:,:,np.newaxis]
    plt.imshow(img_fgd),plt.axis('off'),plt.show()
    
    ## Save the segmented foreground with black background if want to
    # cv2.imwrite(path_original+'/GrabCut_'+file_original, img_fgd)
    
    mask_converted = np.where(mask==1, 0, 1)
    img_bgd = img*mask_converted[:,:,np.newaxis] ## Segmented background for fitting later
    
    
## Part II: Background Fitting (Gaussian)
    
    hist,bin_ = np.histogram(img_bgd.ravel(),bins=range(256))
    hist_bgd = hist[1:] ## Discard the pixels of grayscale=0 as they belongs to the foreground
    bin_bgd = bin_[1:255] ## Discard the last value because bin_ has one more number than hist
    
    ## initial values for fitting, determine by yourself
    peak = np.max(hist_bgd)
    peak_pos = np.argmax(hist_bgd)
    for h in hist_bgd:
        if h >= peak/2:
            half_pos = np.where(hist_bgd==h)
            break
        else:
            continue
    deviation = 2*np.abs((peak_pos-half_pos[0][0]))/2.355

    popt, _ = optimize.curve_fit(gaussian, bin_bgd, hist_bgd, p0=(peak,peak_pos,deviation))

    ## plot the fitting result if want to
    # plt.plot(bin_bgd, hist_bgd)
    # plt.plot(bin_bgd, gaussian(bin_bgd, *popt))
    # plt.show()
    
    if f == 0:
        Background_fit = popt
    else:
        Background_fit = np.vstack((Background_fit, popt))

# Brightness and Contrast Adjustment

In [ ]:
brightness_align = np.min(Background_fit.T[1]) ## Determine the brightness after alignment
contrast_align = np.mean(np.abs(Background_fit.T[2])) ## Determine the contrast after alignment

for f in range(len(files_original)):
    
    file_original = files_original[f]
    imgpath_original = path_original+'/'+file_original
    img = cv2.imread(imgpath_original,0)
    
    mu = Background_fit.T[1][f]
    sigma = Background_fit.T[2][f]
    ## Apply brightness and contrast alignment factors to the original image
    img_aligned = (img-mu*np.ones(img.shape))*contrast_align/sigma + brightness_align*np.ones(img.shape)
    
    ## Cut out values over 255 or below 0 in the 'img_aligned' array, and set them to 0 or 255
    img_clipped = img_aligned.clip(min=0, max=255)
    
    img_save = img_clipped.astype(int)
    
    plt.imshow(img_save,cmap='gray')
    cv2.imwrite(path_original+'/Aligned_'+file_original, img_save)

# Check

In [ ]:
## This part is to check if the brightness and contrast of all images are well aligned

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

path = r'Folder of ALIGNED images'
files = os.listdir(path)

for file in files:
    imgpath = path+'/'+file
    image = cv2.imread(imgpath,0)
    ## GaussianBlur is to make the image less noisy if necessary
    # image = cv2.GaussianBlur(image,(5,5), 0.5)
    
    hist,bin_ = np.histogram(image.ravel(),bins=range(256))
    
    if file == files[0]:
        Hist = np.log(hist+1)
    else:
        Hist = np.vstack((Hist, np.log(hist+1)))

ctf = plt.contourf(Hist)
clb = plt.colorbar(ctf)
plt.show()

## The background pixels should be at the same grayscale level